<a href="https://colab.research.google.com/github/vishnu-096/Gait_Phase_Prediction/blob/main/VAE_gait_phase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import time
import os
import sys
import json
import pickle

import scipy as sp
from scipy import signal

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Conv2D, MaxPool2D, Dropout,Flatten, Conv1D, MaxPool1D
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import metrics
from tensorflow.keras import optimizers, Sequential, Model
from tensorflow.keras.callbacks import LearningRateScheduler

#from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.graphics.tsaplots import plot_acf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import os, warnings, random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import keras.layers as L

from keras import Model
from keras.layers import Layer
from tensorflow.python.keras import backend as K
#from keras.backend import backend as K
#import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

from tensorflow.keras import optimizers, Sequential, Model
from tensorflow.keras.callbacks import LearningRateScheduler
from google.colab import drive
drive.mount('/content/drive')

main_path = '/content/drive/MyDrive/GaitPhase_final_results/'


Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
sub_cnt_file='sub_cnt.pkl'
sub_cnt_f = open(sub_cnt_file, 'wb')
pickle.dump(0, sub_cnt_f)
sub_cnt_f.close()

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

single_leg = True
ver2 = True

hip_only = False

Num GPUs Available:  0


In [ ]:
def encode_phase(df, name):
    phase = df[name].values.tolist()
    x = np.zeros((len(df[name]),1))
    y = np.zeros((len(df[name]),1))
    for i in range (len(phase)):
        phi = math.radians(phase[i])
        x[i] = math.cos(phi)
        y[i] = math.sin(phi)

    name_x = name[0] + '_ph_hip_x'
    name_y = name[0] + '_ph_hip_y'

    df[name_x] = x
    df[name_y] = y

In [ ]:
def encode_gait_percentage(df, name):
    percent = df[name].values.tolist()
    X = np.zeros((len(df[name]),1))
    Y = np.zeros((len(df[name]),1))
    for i in range (len(percent)):
        phi = percent[i] * 2 * math.pi / 100
        X[i] = math.cos(phi)
        Y[i] = math.sin(phi)
    df['X'] = X
    df['Y'] = Y

def convert_data(d_x, d_y, look_back = 10, fore_cast = 5):
    dataX = []
    dataY = []

    for i in range(look_back, len(d_x) - fore_cast):
        dataX.append(d_x[i - look_back: i])
        dataY.append(d_y[i + fore_cast,:])

    return np.array(dataX), np.array(dataY)

def get_train_data_from_df(all_data, test_ratio):
    all_data.info()
    cycle = 0
    cycle_list = []

    start = 0

    for i in range(len(all_data) - 1):
        if (all_data['perc'][i+1] == 0):
            cycle += 1
            cycle_list.append(all_data.iloc[start:i])
            start = i+1

    random.shuffle(cycle_list)
    source_table = pd.concat(cycle_list, axis=0, ignore_index=True)

    # For hip: source_table = source_table.drop(["lgrf", "rgrf", "lcop","rcop"], axis = 1)

    # all features:  "lcop", "rcop", "lgrf", "rgrf", "l_ph_hip", "r_ph_hip", "st_sw_phase_l", "st_sw_phase_r", "st_l_l", "st_l_r", "leg_len", "weight"

    if ver2 :
        source_table = source_table.drop(['l_ph_hip','r_ph_hip'], axis = 1)
        lgrf_col = source_table.pop('lgrf')  # Extract 'lgrf' column
        rgrf_col = source_table.pop('rgrf')  # Extract 'rgrf' column
        leg_len_col = source_table.pop('leg_len')  # Extract 'lgrf' column
        weight_col = source_table.pop('weight')  # Extract 'rgrf' column
        # Concatenate 'lgrf' and 'rgrf' columns at the rightmost end
        if single_leg:
            source_table['lgrf'] = lgrf_col
            source_table = source_table.drop(['st_l_r', 'st_sw_phase_r', 'r_ph_hip_cos', 'r_ph_hip_sin','rcop'], axis = 1)
            if hip_only:
                source_table = source_table.drop(['lcop'], axis = 1)

        else:
            source_table['lgrf'] = lgrf_col
            source_table['rgrf'] = rgrf_col
            if hip_only:
                source_table = source_table.drop(['lcop', 'rcop'], axis = 1)


    else:
        source_table = source_table.drop(["lgrf", "rgrf",'l_ph_hip','r_ph_hip', 'leg_len', 'weight'], axis = 1)
        if single_leg:
            source_table = source_table.drop(['st_l_r', 'st_sw_phase_r', 'r_ph_hip_cos', 'r_ph_hip_sin','rcop'], axis = 1)
            if hip_only:
                source_table = source_table.drop(['lcop'], axis = 1)
        elif hip_only:
            source_table = source_table.drop(['lcop', 'rcop'], axis = 1)
    source_table
    x = source_table
    x = x.drop(['perc'], axis=1)
    # scaler = MinMaxScaler()
    # x_scaled = scaler.fit_transform(x)
    # x_scaled = pd.DataFrame(x_scaled)

    encode_gait_percentage(source_table, 'perc')
    y = source_table[["X","Y"]]
    # data_x = x_scaled
    data_x=x.values
    x.info()
    data_y = y.values.reshape(-1,2)

    X_train, X_test,y_train, y_test = train_test_split(data_x, data_y ,
                            test_size=0.25,
                            shuffle=False)
    look_back = 10
    fore_cast = 1

    train_x, train_y = convert_data(X_train, y_train, look_back, fore_cast)
    validation_x, validation_y = convert_data(X_test, y_test, look_back, fore_cast)

    return train_x, train_y, validation_x, validation_y


import random
def get_data_frames_from_files(path, file_names, subject_dict, subjects):
    file_list=[]
    for i in range (len(file_names)):

        subject = file_names[i].split('_')[0]
        if not subject in subjects:
            continue
        else:
            print(file_names[i])
        leg_len = subject_dict[subject][0]
        weight = subject_dict[subject][1]
        tmp=pd.read_excel(path+ file_names[i], sheet_name='Sheet1')
        perc_column = tmp['perc']
        tmp = tmp.drop(columns=['perc'])
        scaler = MinMaxScaler()

    # Normalize each column separately
        normalized_data = scaler.fit_transform(tmp)
        column_names = tmp.columns

        normalized_df = pd.DataFrame(normalized_data, columns=column_names)
        tmp['l_ph_hip']=tmp['l_ph_hip']/300
        tmp['r_ph_hip']=tmp['r_ph_hip']/300
        tmp['l_ph_fo']=tmp['l_ph_fo']/300
        tmp['r_ph_fo']=tmp['r_ph_fo']/300
        tmp['lcop']= tmp['lcop']*1000
        tmp['rcop']=tmp['rcop']*1000
        tmp['strike_frame']=tmp['strike_frame']/400
        tmp['st_sw_phase']=tmp['st_sw_phase']/200
        normalized_df =tmp
        normalized_df['leg_len']=leg_len
        normalized_df['weight']=weight
        normalized_df['perc']= perc_column
        # normalized_df.insert(tmp.columns.get_loc('col1'), 'perc', perc_column)

        file_list.append(normalized_df)

    random.shuffle(file_list)

    all_data = pd.concat(file_list, axis=0, ignore_index=True)
    return all_data


seq_len=10
n_features=11
#train_x=np.zeros((10,10,11))
#encoder_model().summary()
#decoder_model().summary()

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        curLR = optimizer.lr
        return curLR
    return lr

def custom_loss(y_true, y_pred):
    #angle_true = ((math.atan2(y_true[:,1],y_true[:,0]) + 2*math.pi) % (2*math.pi))
    #angle_pred = ((math.atan2(y_pred[:,1],y_pred[:,0]) + 2*math.pi) % (2*math.pi))

    angle_true = ((K.math_ops.atan2(y_true[:,1],y_true[:,0]) + 2*math.pi) % (2*math.pi))
    angle_pred = ((K.math_ops.atan2(y_pred[:,1],y_pred[:,0]) + 2*math.pi) % (2*math.pi))

    #angle_true = tf.make_ndarray(y_true)
    #angle_pred = tf.make_ndarray(y_pred)

    #print("y_true: ", y_true.size())
    #print("info: ", y_true.info())
    #print("y_pred: ", y_pred.shape())
    #print("y_true:", y_true.shape)
    #a = K.argmax(y_true, axis=1)
    print("ang_pred: ",angle_pred)
    #print(angle_pred.shape)
    loss = tf.keras.losses.mean_squared_error(angle_true, angle_pred)
    return loss

class attention(Layer):
    def __init__(self, reduce = False, **kwargs):
        super(attention,self).__init__(**kwargs)
        self.reduce = reduce
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        print(x.shape)
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        if self.reduce:
            context = K.sum(context, axis=1)
        return context


def train_mlp_model(train_x, train_y, look_back):
    epochs = 30
    batch_size = 32
    learning_rate = 1e-2

    # learning rate scheduler
    use_lr_scheduler = True
    lr_factor = 0.5
    lr_patience = 8
    lr_threshold = 0.01
    lr_min_rate = 1e-6

    n = train_
    if ver2:
        if single_leg:
          train_data = train_y[:,:, :n-1]
        else:
          train_data = train_x[:,:, :n-2]
    else:
        train_data = train_x[:,:,:]

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    mlp_model = Sequential()
    mlp_model.add(Conv1D(100, kernel_size=3, strides=1,input_shape=(look_back, train_x.shape[2]), activation='relu'))
    mlp_model.add(MaxPool1D(pool_size=3, strides=1))
    mlp_model.add(Dropout(0.1))
    mlp_model.add(Conv1D(32, kernel_size = 2, strides = 1, activation='tanh'))
    mlp_model.add(MaxPool1D(pool_size=2, strides=1))
    mlp_model.add(Dropout(0.1))
    #mlp_model.add(Conv1D(8, kernel_size = 2, strides = 1, activation = 'tanh'))
    #mlp_model.add(MaxPool1D(pool_size=2, strides=1))
    #mlp_model.add(Dropout(0.2))
    mlp_model.add(Flatten())
    mlp_model.add(Dense(2))

    print(mlp_model.summary())

    monitor = EarlyStopping(monitor='loss', min_delta=1e-5, patience=2, verbose=1, mode='auto')

    #optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    mlp_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy','mae','mse'])
    history = mlp_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, callbacks=monitor,verbose=1)

    return mlp_model

def train_lstm_ae(train_x):

    lr = 0.001
    epochs = 30
    batch = 128
    n_features=train_x.shape[2]
    train_with_grf = train_x
    n_features=train_x.shape[2]
    if ver2:
        if single_leg:
          train_data = train_x[:,:, :n_features-1]
          n_op_features = train_data.shape[2]+1
        else:
          train_data = train_x[:,:, :n_features-2]
          n_op_features = train_data.shape[2]+2
    else:
        train_data = train_x[:,:,:]
        n_op_features = train_data.shape[2]

    n_features=train_data.shape[2]

    x=Input(shape=(10, n_features))

    # RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    l1=tf.keras.layers.Conv1D(16, 2, padding="same")(x)
    l2=tf.keras.layers.Conv1D(16, 2, padding="same")(l1)
    ###LSTM_layer1 = LSTM(64, return_sequences=True)(l2)
    LSTM_layer2 = LSTM(32, return_sequences=True)(l2)
    attn_layer1 = attention(True)(LSTM_layer2)
    rep_layer = L.RepeatVector((10))(attn_layer1)
    # rep_layer = attn_layer1
    # attn_layer2 = attention()(rep_layer)
    LSTM_layer3 = LSTM(32, return_sequences=True)(rep_layer)
    LSTM_layer4 = LSTM(64, return_sequences=True)(LSTM_layer3)
    lin_layer = L.TimeDistributed(L.Dense(n_op_features))(LSTM_layer4)
    # tmp_layer = Flatten()(rep_layer)
    # lin_layer = L.Dense(2)(tmp_layer)
    model=tf.keras.Model(x,lin_layer)
    model.summary()
    adam = optimizers.Adam(lr)
    model.compile(loss='mse', optimizer=adam)

    model_history = model.fit( train_data, train_with_grf, epochs=epochs, batch_size=batch,  verbose=1)

    encoder = tf.keras.Model(inputs=model.inputs, outputs=model.layers[3].output)
    train_encoded = encoder.predict(train_data)
    print('Encoded time-series shape', train_encoded.shape)
    print('Encoded time-series sample', train_encoded[0])
    # validation_encoded = encoder.predict(validation_x)

    return train_encoded, encoder, model

def train_mlp_model_ae(train_encoded, train_y):
    batch=128
    mlp_model = Sequential()
    mlp_model.add(tf.keras.Input(shape=(train_encoded.shape[1], train_encoded.shape[2]), name='input_layer'))
    mlp_model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=2, strides=1, activation='relu', padding='same', input_dim = (train_encoded.shape[1])))
    mlp_model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same'))

    mlp_model.add(LSTM(16))

    mlp_model.add(L.Dense(32, kernel_initializer='glorot_normal', activation='relu'))
    # mlp_model.add(L.Dense(32, kernel_initializer='glorot_normal', activation='relu', input_dim=(train_encoded.shape[1])))

    mlp_model.add(L.Dense(8, kernel_initializer='glorot_normal', activation='relu'))
    mlp_model.add(L.Dense(2))
    mlp_model.summary()

    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)
    adam = optimizers.Adam(lr_schedule)

    mlp_model.compile(loss='mse', optimizer=adam)
    lrate = LearningRateScheduler(step_decay)
    epochs = 30
    # lrate = LearningRateScheduler(step_decay)
    monitor = EarlyStopping(monitor='loss', min_delta=1e-5, patience=3, verbose=1, mode='auto')
    # train_encoded_reshaped=np.reshape(train_encoded,(train_encoded.shape[0], train_encoded.shape[1]*train_encoded.shape[2]))

    # train_encoded_reshaped=np.reshape(train_encoded,(train_encoded.shape[0], train_encoded.shape[1]))
    train_encoded_reshaped = train_encoded
    callback_list = [monitor]
    mlp_history = mlp_model.fit(train_encoded_reshaped , train_y, callbacks=callback_list, epochs=epochs, batch_size=batch,  verbose=2)

    return mlp_model


def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.005
    epochs_drop = 10
    return initial_lrate - 0.00001

def test_model_get_results(encoder, mlp_model, validation_x, validation_y, display_flag, tag, file):

    n_features = validation_x.shape[2]
    if ver2:
        if single_leg:
          test_data = validation_x[:,:, :n_features-1]
          n_op_features = test_data.shape[2]+1
        else:
          test_data = validation_x[:,:, :n_features-2]
          n_op_features = test_data.shape[2]+2
    else:
        test_data = validation_x[:,:,:]
        n_op_features = test_data.shape[2]

    ls_v = encoder.predict(test_data)


    print("validation_x shape",test_data.shape)
    testPredict = mlp_model.predict(ls_v)
    print(validation_y.shape)
    print(testPredict.shape)
    testScore = np.sqrt(mean_squared_error(validation_y, testPredict))
    # print(testScore)
    # print('Test Score: %.2f RMSE' % (testScore))
    pred = np.zeros((len(testPredict),1))
    ang_pred = np.zeros((len(testPredict),1))

    for iter in range(len(testPredict)):
        x = testPredict[iter][0]
        y = testPredict[iter][1]
        pred[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi)) * (100 / (2*math.pi))
        ang_pred[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi))

    actual = np.zeros((len(validation_y),1))
    ang_act = np.zeros((len(validation_y),1))

    for iter in range(len(validation_y)):
        x =validation_y[iter][0]
        y =validation_y[iter][1]
        # st_frame = validation_x[0,9,7]

        actual[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi)) * (100 / (2*math.pi))
        ang_act[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi))
        # if st_frame>100 and pred[iter]<15:
        #     pred[iter]=100-pred[iter]""
        # if st_frame<90 and pred[iter]>85:
        #     pred[iter]=100-pred[iter]


    cor_actual=[]
    cor_pred=[]
    prec_list=[]

    for i in range(5):
        correct = 0
        for iter in range(len(actual)):
            # if (actual[iter]>98) or(actual[iter]<3):
            #     correct+=1
            #     continue
            if (abs(actual[iter] - pred[iter]) <= (i+1)):
                correct+=1
            cor_pred.append(pred[iter])
            cor_actual.append(actual[iter])
        prec=correct * 100/len(actual)
        print("Precision ", i+1, ": ", prec)
        # file.write(str(prec))
        # file.write("\n")
        prec_list.append(prec)

    rmse = 0
    length = len(actual)
    for i in range(len(actual)):
        if abs (pred[i] - actual[i]) >=90:
            length -= 1
        else:
            rmse = rmse + pow(pred[i] - actual[i], 2)
    rmse = rmse / length
    rmse = math.sqrt(rmse)

    if os.path.exists(result_pkl_file):
        with open(result_pkl_file,'rb') as pkl_file:
            res_dict=pickle.load(pkl_file)
    else:
        res_dict={}
    result={}
    result['rmse']=rmse
    result['prec_list']=prec_list
    res_dict[tag]=result
    with open(result_pkl_file, 'wb')as pkl_file:
        pickle.dump(res_dict, pkl_file)


    print(rmse)
    # file.write("rmse "+str(rmse))
    # file.write("\n")
    if display_flag:
        plt.scatter(cor_actual, cor_pred, facecolors='none', edgecolors='crimson',alpha=0.4)
        p1 = max(max(cor_pred), max(cor_actual))
        p2 = min(min(cor_pred), min(cor_actual))

        ci = 0.1 * np.std([p1,p2]) / np.mean([p1,p2])

        plt.plot([p1, p2], [p1, p2], 'b-', linewidth =3)
        plt.title('Actual vs Prediction')
        plt.savefig(result_path+tag+"__res.png")

    return prec_list, rmse


In [ ]:
file_names = ['JL_I_0_new_.xlsx', 'JL_I_2_new_.xlsx','JL_I_3_new_.xlsx','JL_I_5_new_.xlsx','JL_I_4_new_.xlsx',
              'JS_I_1_new_.xlsx', 'JS_I_2_new_.xlsx','JS_I_3_new_.xlsx','JS_I_5_new_.xlsx','JS_I_4_new_.xlsx',
              'AK_I_0_new_.xlsx', 'AK_I_2_new_.xlsx','AK_I_3_new_.xlsx','AK_I_5_new_.xlsx','AK_I_4_new_.xlsx',
              'VN_I_0_new_.xlsx', 'VN_I_2_new_.xlsx','VN_I_3_new_.xlsx','VN_I_5_new_.xlsx','VN_I_4_new_.xlsx',
              'VP_I_0_new_.xlsx', 'VP_I_2_new_.xlsx','VP_I_3_new_.xlsx','VP_I_5_new_.xlsx','VP_I_4_new_.xlsx',
             'SOE_I_0_new_.xlsx', 'SOE_I_2_new_.xlsx','SOE_I_3_new_.xlsx','SOE_I_5_new_.xlsx', 'SOE_I_4_new_.xlsx', 'SD_I_3_new_.xlsx', 'SD_I_4_new_.xlsx','SD_I_5_new_.xlsx',
             'SD_I_1_new_.xlsx','SD_I_2_new_.xlsx','TH_I_0_new_.xlsx', 'TH_I_2_new_.xlsx', 'TH_I_3_new_.xlsx','TH_I_4_new_.xlsx', 'TH_I_5_new_.xlsx'
             ,'PK_I_0_new_.xlsx', 'PK_I_2_new_.xlsx', 'PK_I_3_new_.xlsx','PK_I_5_new_.xlsx',
              'SKS_0_I_new_.xlsx', 'SKS_2_I_new_.xlsx','SKS_3_I_new_.xlsx','SKS_4_I_new_.xlsx','SKS_5_I_new_.xlsx',
            'PH_I_0_new_.xlsx',  'PH_I_2_new_.xlsx',  'PH_I_3_new_.xlsx',  'PH_I_4_new_.xlsx',  'PH_I_5_new_.xlsx',
            'YC_I_0_new_.xlsx',  'YC_I_2_new_.xlsx',  'YC_I_3_new_.xlsx',  'YC_I_4_new_.xlsx',  'YC_I_5_new_.xlsx'
              ]
subject_dict = {'VN':[0.90,0.63],'AK':[0.80,0.57],'JS':[0.89,0.64],'JL':[0.79,0.63],'SKS':[0.83, 0.58],'VP':[0.93, 0.77],'SOE':[0.90, 0.83],
                'SD':[0.83, 0.70], 'TH':[0.66, 0.52], 'PK':[0.90, 0.88], 'PH':[0.95,0.80], 'YC':[0.82,0.79]}
subject_names = ['VP','AK', 'SD','PH', 'YC','JL','JS', 'PK', 'SOE', 'TH','SKS', 'VN']#,'VN','AK' 'SOE'
sub_comb_list=[]
test_sub_list=[]
acc_list=[]
rmse_list=[]
test_acc_list=[]
test_rmse_list=[]


#path="/home/vtp/Gait_Phase_Prediction/Subject_data/Final_files/"
#result_path = "/home/vtp/Gait_Phase_Prediction/Results/final/"

path = main_path
result_path = main_path

result_pkl_file=result_path+'results_ae_hip_cop_single.pkl'
pkl_file=path+"all_sub_vae_ver_hip_final.pkl"
# pkl_file=path+"good_sub_data.pkl"

for sub in subject_names:

  test_sub_list.append(sub)
  tmp=subject_names.copy()
  tmp.remove(sub)
  sub_comb_list.append(tmp)

df_dict={}

with open(pkl_file, 'rb') as file:
    df_dict = pickle.load(file)


sub_cnt_file='sub_cnt.pkl'

with open(result_path+"all_results_cnn_both_hip.txt","w") as file:
    for sub_iter, sub_comb in enumerate(sub_comb_list):
    #for a_ in range(len(subject_names)):
        sub_file=open(sub_cnt_file,'rb')
        cnt=pickle.load(sub_file)
        sub_file.close()
        if cnt==len(subject_names):
           print("Done with all subjects !!")
           break
        sub_comb=sub_comb_list[cnt]
        sub_iter=cnt
        print("Subject combination :",sub_comb)
        sub_tag=test_sub_list[sub_iter]
        # all_data=get_data_frames_from_files(path, file_names, subject_dict, sub_comb)
        file_list=[]
        for sub in sub_comb:
          tmp_df=df_dict[sub]
          file_list.append(tmp_df)
        random.shuffle(file_list)

        all_data = pd.concat(file_list, axis=0, ignore_index=True)
        print("All Data Info: ")
        all_data.info()
        train_x, train_y, validation_x, validation_y = get_train_data_from_df(all_data, 0.25)
        print("Nan in train x at indices:",np.argwhere(np.isnan(train_x)))
        print("Nan in train y at indices:",np.argwhere(np.isnan(train_y)))

        print(train_x.shape)
        print(validation_x.shape)
        print("SSS")
        seq_len = train_x.shape[1]
        n_features = train_x.shape[2]

        mlp_model=[]
        print("OKAY till here 1")

        look_back = 10
        print(train_y.shape)

        samp_t, encoder, model = train_lstm_ae(train_x)

        mlp_model = train_mlp_model_ae(samp_t, train_y)

        # mlp_model = train_mlp_model(train_x, train_y, look_back)
        # file.write("Training Result :")
        # file.write("\n")
        # acc, rmse=test_model_get_results(encoder, mlp_model, validation_x, validation_y, False, sub_tag, file)
        # acc_list.append(acc)
        # rmse_list.append(rmse)
        print("testing on :", test_sub_list[sub_iter])
        file.write("Testing  Result :")
        file.write(test_sub_list[sub_iter])
        file.write("\n")
        # all_data=get_data_frames_from_files(path, file_names, subject_dict, [test_sub_list[sub_iter]])
        all_data = df_dict[test_sub_list[sub_iter]]
        test_x, test_y, validation_x, validation_y = get_train_data_from_df(all_data, 0.25)
        acc, rmse=test_model_get_results(encoder, mlp_model, test_x, test_y, True, sub_tag, file)
        # tf.keras.backend.clear_session()

        test_acc_list.append(acc)
        test_rmse_list.append(rmse)
        # break
        # file.write(acc)
        file.write("\n")
        cnt=cnt+1
        sub_file=open(sub_cnt_file,'wb')
        pickle.dump(cnt, sub_file)
        sub_file.close()
        # if sub_iter>=5:
        # break
    file.close()


Subject combination : ['AK', 'SD', 'PH', 'YC', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572285 entries, 0 to 572284
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           572285 non-null  float64
 1   rcop           572285 non-null  float64
 2   lgrf           572285 non-null  float64
 3   rgrf           572285 non-null  float64
 4   l_ph_hip       572285 non-null  float64
 5   r_ph_hip       572285 non-null  float64
 6   l_ph_hip_cos   572285 non-null  float64
 7   l_ph_hip_sin   572285 non-null  float64
 8   r_ph_hip_cos   572285 non-null  float64
 9   r_ph_hip_sin   572285 non-null  float64
 10  st_sw_phase_l  572285 non-null  float64
 11  st_sw_phase_r  572285 non-null  float64
 12  st_l_l         572285 non-null  float64
 13  st_l_r         572285 non-null  float64
 14  leg_len        572285 non-null  float64
 15  weight      

<ipython-input-20-6dbb38482f3c>:424: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


2.217286201097019
Subject combination : ['VP', 'SD', 'PH', 'YC', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569770 entries, 0 to 569769
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           569770 non-null  float64
 1   rcop           569770 non-null  float64
 2   lgrf           569770 non-null  float64
 3   rgrf           569770 non-null  float64
 4   l_ph_hip       569770 non-null  float64
 5   r_ph_hip       569770 non-null  float64
 6   l_ph_hip_cos   569770 non-null  float64
 7   l_ph_hip_sin   569770 non-null  float64
 8   r_ph_hip_cos   569770 non-null  float64
 9   r_ph_hip_sin   569770 non-null  float64
 10  st_sw_phase_l  569770 non-null  float64
 11  st_sw_phase_r  569770 non-null  float64
 12  st_l_l         569770 non-null  float64
 13  st_l_r         569770 non-null  float64
 14  leg_len        569770 non-null  float64

<ipython-input-20-6dbb38482f3c>:424: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


1.6457412981504247
Subject combination : ['VP', 'AK', 'PH', 'YC', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579277 entries, 0 to 579276
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           579277 non-null  float64
 1   rcop           579277 non-null  float64
 2   lgrf           579277 non-null  float64
 3   rgrf           579277 non-null  float64
 4   l_ph_hip       579277 non-null  float64
 5   r_ph_hip       579277 non-null  float64
 6   l_ph_hip_cos   579277 non-null  float64
 7   l_ph_hip_sin   579277 non-null  float64
 8   r_ph_hip_cos   579277 non-null  float64
 9   r_ph_hip_sin   579277 non-null  float64
 10  st_sw_phase_l  579277 non-null  float64
 11  st_sw_phase_r  579277 non-null  float64
 12  st_l_l         579277 non-null  float64
 13  st_l_r         579277 non-null  float64
 14  leg_len        579277 non-null  float6

...

1.6188963325024763
Subject combination : ['VP', 'AK', 'SD', 'YC', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567615 entries, 0 to 567614
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           567615 non-null  float64
 1   rcop           567615 non-null  float64
 2   lgrf           567615 non-null  float64
 3   rgrf           567615 non-null  float64
 4   l_ph_hip       567615 non-null  float64
 5   r_ph_hip       567615 non-null  float64
 6   l_ph_hip_cos   567615 non-null  float64
 7   l_ph_hip_sin   567615 non-null  float64
 8   r_ph_hip_cos   567615 non-null  float64
 9   r_ph_hip_sin   567615 non-null  float64
 10  st_sw_phase_l  567615 non-null  float64
 11  st_sw_phase_r  567615 non-null  float64
 12  st_l_l         567615 non-null  float64
 13  st_l_r         567615 non-null  float64
 14  leg_len        567615 non-null  float6

<ipython-input-20-6dbb38482f3c>:424: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


2.091890927820046
Subject combination : ['VP', 'AK', 'SD', 'PH', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577457 entries, 0 to 577456
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           577457 non-null  float64
 1   rcop           577457 non-null  float64
 2   lgrf           577457 non-null  float64
 3   rgrf           577457 non-null  float64
 4   l_ph_hip       577457 non-null  float64
 5   r_ph_hip       577457 non-null  float64
 6   l_ph_hip_cos   577457 non-null  float64
 7   l_ph_hip_sin   577457 non-null  float64
 8   r_ph_hip_cos   577457 non-null  float64
 9   r_ph_hip_sin   577457 non-null  float64
 10  st_sw_phase_l  577457 non-null  float64
 11  st_sw_phase_r  577457 non-null  float64
 12  st_l_l         577457 non-null  float64
 13  st_l_r         577457 non-null  float64
 14  leg_len        577457 non-null  float64

<ipython-input-20-6dbb38482f3c>:424: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


4.920920170270712
Subject combination : ['VP', 'AK', 'SD', 'PH', 'YC', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581777 entries, 0 to 581776
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           581777 non-null  float64
 1   rcop           581777 non-null  float64
 2   lgrf           581777 non-null  float64
 3   rgrf           581777 non-null  float64
 4   l_ph_hip       581777 non-null  float64
 5   r_ph_hip       581777 non-null  float64
 6   l_ph_hip_cos   581777 non-null  float64
 7   l_ph_hip_sin   581777 non-null  float64
 8   r_ph_hip_cos   581777 non-null  float64
 9   r_ph_hip_sin   581777 non-null  float64
 10  st_sw_phase_l  581777 non-null  float64
 11  st_sw_phase_r  581777 non-null  float64
 12  st_l_l         581777 non-null  float64
 13  st_l_r         581777 non-null  float64
 14  leg_len        581777 non-null  float64